In [ ]:
from __future__ import annotations

from pathlib import Path
from datetime import date
from collections import Counter
import glob
import gc

import polars as pl
import matplotlib.pyplot as plt


In [2]:
# -----------------------------
# Paths / config
# -----------------------------
PARQUET_DIR = Path("/media/vatereal/Main/parquet")
OUTPUT_DIR = Path("/media/vatereal/Main/outputs")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

patterns = {
    "blocks": str(PARQUET_DIR / "blocks/day=*/blocks-*.parquet"),
    "txs":    str(PARQUET_DIR / "txs/day=*/txs-*.parquet"),
    "io":     str(PARQUET_DIR / "io/day=*/io-*.parquet"),
}

pl.Config.set_tbl_rows(20)
pl.Config.set_fmt_str_lengths(80)

print("Parquet file counts:", {k: len(glob.glob(v)) for k, v in patterns.items()})

CUTOFF_DAY = date(2013, 1, 1)

Parquet file counts: {'blocks': 7713, 'txs': 7713, 'io': 7678}


In [3]:
# -----------------------------
# Helpers
# -----------------------------
def extract_day_from_path(path: str) -> date | None:
    """
    Given a path like /.../io/day=2013-05-01/io-0001.parquet
    return date(2013, 5, 1).
    """
    p = Path(path)
    for part in p.parts:
        if part.startswith("day="):
            day_str = part.split("=", 1)[1]
            try:
                return date.fromisoformat(day_str)
            except ValueError:
                return None
    return None


class UnionFind:
    def __init__(self):
        self.parent: list[int] = []
        self.rank: list[int] = []

    def make_set(self) -> int:
        idx = len(self.parent)
        self.parent.append(idx)
        self.rank.append(0)
        return idx

    def find(self, x: int) -> int:
        parent = self.parent
        while parent[x] != x:
            parent[x] = parent[parent[x]]
            x = parent[x]
        return x

    def union(self, x: int, y: int) -> None:
        rx = self.find(x)
        ry = self.find(y)
        if rx == ry:
            return

        parent = self.parent
        rank = self.rank

        if rank[rx] < rank[ry]:
            parent[rx] = ry
        elif rank[rx] > rank[ry]:
            parent[ry] = rx
        else:
            parent[ry] = rx
            rank[rx] += 1


def addr_type(addr: str) -> str:
    """Rough address type from prefix (legacy P2PKH, P2SH, Bech32, Taproot, etc.)."""
    if addr.startswith("1"):
        return "p2pkh"
    if addr.startswith("3"):
        return "p2sh"
    if addr.startswith("bc1q"):
        return "bech32_p2wpkh"
    if addr.startswith("bc1p"):
        return "taproot"
    return "other"


def detect_coinjoin_like(n_in: int, out_values: list[float]) -> bool:
    """
    Very simple CoinJoin-ish heuristic:
    - at least 3 inputs
    - at least 3 outputs
    - at least 3 outputs with exactly the same value
    """
    n_out = len(out_values)
    if n_in < 3 or n_out < 3:
        return False
    cnt = Counter(out_values)
    max_eq = max(cnt.values())
    return max_eq >= 3


In [4]:
# -----------------------------
# Init structures
# -----------------------------
io_paths = sorted(glob.glob(patterns["io"]))
print(f"Found {len(io_paths)} io parquet files.")

uf = UnionFind()
addr_to_id: dict[str, int] = {}  # address -> union-find node index

# RAM-friendly flags indexed by node_id:
# seen_output_flags[i] == 1 if address i was ever seen as a vout
# multi_change_flags[i] bit 0 -> multi-input, bit 1 -> change
seen_output_flags = bytearray()
multi_change_flags = bytearray()

# For coverage / stats
n_txs_total = 0
n_txs_with_multiinput = 0
n_txs_coinjoin_flagged = 0
n_txs_with_change_detected = 0

Found 7678 io parquet files.


In [5]:
# -----------------------------
# Build final entity mapping
# -----------------------------
print("Finalizing entity mapping (compressing components)...")

# Map each node to its root
node_to_root: dict[int, int] = {}
for addr, node_id in addr_to_id.items():
    root = uf.find(node_id)
    node_to_root[node_id] = root

# Map each root to a compact entity_id (0..N-1)
root_to_entity: dict[int, int] = {}
next_entity_id = 0
entity_ids: list[int] = []

for addr, node_id in addr_to_id.items():
    root = node_to_root[node_id]
    ent = root_to_entity.get(root)
    if ent is None:
        ent = next_entity_id
        root_to_entity[root] = ent
        next_entity_id += 1
    entity_ids.append(ent)

print(f"Number of unique addresses (2013+): {len(addr_to_id)}")
print(f"Number of entities (clusters): {next_entity_id}")

Finalizing entity mapping (compressing components)...
Number of unique addresses (2013+): 0
Number of entities (clusters): 0


In [6]:
entities_df = pl.DataFrame(
    {
        "address": list(addr_to_id.keys()),
        "entity_id": entity_ids,
    }
)

out_path = OUTPUT_DIR / "entities_multiinput_change_singleton_2013_onward.parquet"
entities_df.write_parquet(out_path)
print(f"Saved entity mapping to: {out_path}")


Saved entity mapping to: /media/vatereal/Main/outputs/entities_multiinput_change_singleton_2013_onward.parquet


In [7]:
def ensure_flag_capacity(idx: int) -> None:
    """
    Make sure flag arrays are at least as long as needed for node_id idx.
    """
    needed = idx + 1
    cur = len(seen_output_flags)
    if cur < needed:
        delta = needed - cur
        seen_output_flags.extend(b"\x00" * delta)
        multi_change_flags.extend(b"\x00" * delta)


def get_addr_id(addr: str) -> int:
    """Map address string to union-find index (create if new, plus flag slots)."""
    idx = addr_to_id.get(addr)
    if idx is None:
        idx = uf.make_set()
        addr_to_id[addr] = idx
        ensure_flag_capacity(idx)
    return idx


In [ ]:

# -----------------------------
# Main pass over IO parquet files
# -----------------------------
for i, path in enumerate(io_paths, start=1):
    file_day = extract_day_from_path(path)
    if file_day is None:
        print(f"[{i}/{len(io_paths)}] WARNING: could not extract day from path, skipping: {path}")
        continue

    # Skip data before 2013-01-01
    if file_day < CUTOFF_DAY:
        continue

    print(f"[{i}/{len(io_paths)}] Processing {path} (day={file_day}) ...")

    # Read only columns we need (saves RAM)
    df = pl.read_parquet(path, columns=["dir", "txid", "address", "value"])

    # Filter only non-null addresses
    df = (
        df
        .filter(pl.col("address").is_not_null())
        .select(["dir", "txid", "address", "value"])
    )

    if df.height == 0:
        del df
        gc.collect()
        continue

    # --- Ensure every address gets a node_id (singleton by default) ---
    unique_addrs = df.select(pl.col("address").unique())["address"].to_list()
    for a in unique_addrs:
        get_addr_id(a)
    del unique_addrs
    gc.collect()

    # Split into inputs and outputs
    vin_df = (
        df
        .filter(pl.col("dir") == "vin")
        .group_by("txid")
        .agg(
            pl.col("address").unique().alias("in_addrs")
        )
    )

    vout_df = (
        df
        .filter(pl.col("dir") == "vout")
        .group_by("txid")
        .agg(
            pl.col("address").alias("out_addrs"),   # keep duplicates if same address repeated
            pl.col("value").alias("out_values")
        )
    )

    # df no longer needed
    del df
    gc.collect()

    # Join inputs & outputs per txid
    tx_df = vin_df.join(vout_df, on="txid", how="inner")

    # free vin/vout to save RAM
    del vin_df, vout_df
    gc.collect()

    if tx_df.height == 0:
        del tx_df
        gc.collect()
        continue

    # Iterate over transactions
    for row in tx_df.iter_rows(named=True):
        in_addrs: list[str] = row["in_addrs"]
        out_addrs: list[str] = row["out_addrs"]
        out_values: list[float] = row["out_values"]

        if not in_addrs or not out_addrs:
            continue

        n_txs_total += 1

        n_in = len(in_addrs)
        n_out = len(out_addrs)

        # CoinJoin/mixer-like detection: skip these txs for heuristics
        is_coinjoin = detect_coinjoin_like(n_in, out_values)
        if is_coinjoin:
            n_txs_coinjoin_flagged += 1
            # still mark outputs as seen
            for a in out_addrs:
                a_id = get_addr_id(a)
                seen_output_flags[a_id] = 1
            continue

        # --- Multi-input heuristic ---
        if n_in >= 2:
            n_txs_with_multiinput += 1

            in_ids = [get_addr_id(a) for a in in_addrs]
            for idx in in_ids:
                multi_change_flags[idx] |= 1  # mark as multi-input

            first_id = in_ids[0]
            for idx in in_ids[1:]:
                uf.union(first_id, idx)

        # --- Change-address heuristic ---
        if n_in >= 1 and n_out >= 2:
            in_types = [addr_type(a) for a in in_addrs]
            type_counts = Counter(in_types)
            majority_type = max(type_counts, key=type_counts.get)

            candidates: list[str] = []
            for a in out_addrs:
                if a in in_addrs:
                    continue
                if addr_type(a) != majority_type:
                    continue
                a_id = get_addr_id(a)
                if seen_output_flags[a_id]:
                    # stricter: favor new output addresses
                    continue
                candidates.append(a)

            if len(candidates) == 1:
                change_addr = candidates[0]
                change_id = get_addr_id(change_addr)
                multi_change_flags[change_id] |= 2  # mark as change

                n_txs_with_change_detected += 1

                first_input_id = get_addr_id(in_addrs[0])
                uf.union(first_input_id, change_id)

        # mark all outputs as seen for future change detection
        for a in out_addrs:
            a_id = get_addr_id(a)
            seen_output_flags[a_id] = 1

    # free tx_df at end of this file
    del tx_df
    gc.collect()


[1498/7678] Processing /media/vatereal/Main/parquet/io/day=2013-01-01/io-000214563-000214724.parquet (day=2013-01-01) ...
[1499/7678] Processing /media/vatereal/Main/parquet/io/day=2013-01-02/io-000214725-000214877.parquet (day=2013-01-02) ...
[1500/7678] Processing /media/vatereal/Main/parquet/io/day=2013-01-03/io-000214878-000215032.parquet (day=2013-01-03) ...
[1501/7678] Processing /media/vatereal/Main/parquet/io/day=2013-01-03/io-000215033-000215039.parquet (day=2013-01-03) ...
[1502/7678] Processing /media/vatereal/Main/parquet/io/day=2013-01-04/io-000215040-000215192.parquet (day=2013-01-04) ...
[1503/7678] Processing /media/vatereal/Main/parquet/io/day=2013-01-05/io-000215193-000215333.parquet (day=2013-01-05) ...
[1504/7678] Processing /media/vatereal/Main/parquet/io/day=2013-01-06/io-000215334-000215478.parquet (day=2013-01-06) ...
[1505/7678] Processing /media/vatereal/Main/parquet/io/day=2013-01-07/io-000215479-000215646.parquet (day=2013-01-07) ...
[1506/7678] Processing /

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7ec6b8103740>>
Traceback (most recent call last):
  File "/home/vatereal/.cache/pypoetry/virtualenvs/blockchain-MbR7oPYh-py3.12/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 781, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


[2368/7678] Processing /media/vatereal/Main/parquet/io/day=2015-04-23/io-000353304-000353433.parquet (day=2015-04-23) ...
[2369/7678] Processing /media/vatereal/Main/parquet/io/day=2015-04-24/io-000353434-000353557.parquet (day=2015-04-24) ...
[2370/7678] Processing /media/vatereal/Main/parquet/io/day=2015-04-25/io-000353558-000353720.parquet (day=2015-04-25) ...
[2371/7678] Processing /media/vatereal/Main/parquet/io/day=2015-04-26/io-000353721-000353864.parquet (day=2015-04-26) ...
[2372/7678] Processing /media/vatereal/Main/parquet/io/day=2015-04-27/io-000353865-000354001.parquet (day=2015-04-27) ...
[2373/7678] Processing /media/vatereal/Main/parquet/io/day=2015-04-28/io-000354002-000354154.parquet (day=2015-04-28) ...
[2374/7678] Processing /media/vatereal/Main/parquet/io/day=2015-04-29/io-000354155-000354281.parquet (day=2015-04-29) ...
[2375/7678] Processing /media/vatereal/Main/parquet/io/day=2015-04-30/io-000354282-000354415.parquet (day=2015-04-30) ...
[2376/7678] Processing /

In [ ]:
# -----------------------------
# Build final entity mapping
# -----------------------------
print("Finalizing entity mapping (compressing components)...")

n_nodes = len(addr_to_id)
print(f"Number of unique addresses (2013+): {n_nodes}")

if n_nodes == 0:
    print("No addresses found after 2013-01-01. Nothing to cluster.")
else:
    # Map each node -> entity id via connected components
    node_to_entity = [0] * n_nodes
    root_to_entity: dict[int, int] = {}
    next_entity_id = 0

    for node in range(n_nodes):
        root = uf.find(node)
        ent = root_to_entity.get(root)
        if ent is None:
            ent = next_entity_id
            root_to_entity[root] = ent
            next_entity_id += 1
        node_to_entity[node] = ent

    print(f"Number of entities (clusters): {next_entity_id}")

    # Compute cluster sizes directly from node_to_entity
    cluster_size_counter = Counter(node_to_entity)
    cluster_sizes = list(cluster_size_counter.values())

    n_entities = len(cluster_size_counter)
    n_singletons = sum(1 for s in cluster_sizes if s == 1)
    n_large = sum(1 for s in cluster_sizes if s >= 10)

    # Build output lists (address, entity_id)
    addresses_out: list[str] = []
    entity_ids_out: list[int] = []

    for addr, node_id in addr_to_id.items():
        addresses_out.append(addr)
        entity_ids_out.append(node_to_entity[node_id])

    entities_df = pl.DataFrame(
        {
            "address": addresses_out,
            "entity_id": entity_ids_out,
        }
    )

    out_path = OUTPUT_DIR / "entities_multiinput_change_singleton_2013_onward.parquet"
    entities_df.write_parquet(out_path)
    print(f"Saved entity mapping to: {out_path}")

    # We no longer need the address dict; other large structures will be GC'ed soon anyway
    del addr_to_id
    gc.collect()

    # -----------------------------
    # Basic stats & plots
    # -----------------------------
    # Heuristic coverage (address-level)
    n_addrs_total = n_nodes
    # For safety, only consider first n_nodes entries in the flag arrays
    flags_view = multi_change_flags[:n_nodes]

    n_addrs_multi = sum(1 for v in flags_view if (v & 1))
    n_addrs_change = sum(1 for v in flags_view if (v & 2))
    n_addrs_touched = sum(1 for v in flags_view if (v & 3))
    n_addrs_only_singleton = n_addrs_total - n_addrs_touched

    print("Computing cluster size statistics...")

    print(f"Total entities: {n_entities}")
    print(f"Singleton entities (size=1): {n_singletons}")
    print(f"Entities with size >= 10: {n_large}")

    print("\nHeuristic coverage:")
    print(f"  Total txs processed (with both vin & vout): {n_txs_total}")
    print(f"  Txs flagged as CoinJoin-like (skipped): {n_txs_coinjoin_flagged}")
    print(f"  Txs using multi-input heuristic: {n_txs_with_multiinput}")
    print(f"  Txs with change-address detected: {n_txs_with_change_detected}")
    print(f"  Addresses in multi-input txs (unique): {n_addrs_multi}")
    print(f"  Addresses detected as change (unique): {n_addrs_change}")
    print(f"  Addresses only in singleton entities: {n_addrs_only_singleton}")


In [ ]:
    # ---- Plot 1: cluster size distribution ----
    if any(s > 0 for s in cluster_sizes):
        plt.figure(figsize=(8, 5))
        plt.hist(cluster_sizes, bins=100, log=True)
        plt.xlabel("Cluster size (number of addresses in entity)")
        plt.ylabel("Frequency (log scale)")
        plt.title("Distribution of Bitcoin Entity Cluster Sizes (2013+)")
        plt.tight_layout()
        plt.show()
        plt.close()
    else:
        print("No positive cluster sizes to plot (cluster size histogram skipped).")

    # ---- Plot 2: heuristic coverage (address-level) ----
    if n_addrs_total > 0:
        plt.figure(figsize=(6, 4))
        plt.bar(
            ["multi-input\n(addrs)", "change\n(addrs)", "only\nsingleton"],
            [n_addrs_multi, n_addrs_change, n_addrs_only_singleton],
        )
        plt.ylabel("Number of addresses")
        plt.title("Heuristic Coverage of Addresses (2013+)")
        plt.tight_layout()
        plt.show()
        plt.close()
    else:
        print("No addresses to plot (heuristic coverage bar chart skipped).")


In [ ]:
from __future__ import annotations

from pathlib import Path
from datetime import date
from collections import Counter
import glob
import gc  # for explicit garbage collection

import polars as pl
import matplotlib.pyplot as plt

# -----------------------------
# Paths / config
# -----------------------------
PARQUET_DIR = Path("/media/vatereal/Main/parquet")
OUTPUT_DIR = Path("/media/vatereal/Main/outputs")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

patterns = {
    "blocks": str(PARQUET_DIR / "blocks/day=*/blocks-*.parquet"),
    "txs":    str(PARQUET_DIR / "txs/day=*/txs-*.parquet"),
    "io":     str(PARQUET_DIR / "io/day=*/io-*.parquet"),
}

pl.Config.set_tbl_rows(20)
pl.Config.set_fmt_str_lengths(80)

print("Parquet file counts:", {k: len(glob.glob(v)) for k, v in patterns.items()})

# Only cluster for data on/after this day
CUTOFF_DAY = date(2013, 1, 1)

# Treat these dir values as outputs; everything else will be treated as inputs
OUTPUT_DIR_VALUES = {"vout", "out", "output", "o"}

# Whether to create a union-find node for every address seen
# (even if that address never appears in a multi-input/change tx).
# This guarantees you get a mapping for all addresses, at the cost of more RAM.
PRECREATE_NODES_FOR_ALL_ADDRS = True

# For plotting, don't try to histogram if there are too many entities
MAX_ENTITIES_FOR_PLOT = 5_000_000

# -----------------------------
# Helpers
# -----------------------------
def extract_day_from_path(path: str) -> date | None:
    """
    Given a path like /.../io/day=2013-05-01/io-0001.parquet
    return date(2013, 5, 1).
    """
    p = Path(path)
    for part in p.parts:
        if part.startswith("day="):
            day_str = part.split("=", 1)[1]
            try:
                return date.fromisoformat(day_str)
            except ValueError:
                return None
    return None


class UnionFind:
    def __init__(self):
        self.parent: list[int] = []
        self.rank: list[int] = []

    def make_set(self) -> int:
        idx = len(self.parent)
        self.parent.append(idx)
        self.rank.append(0)
        return idx

    def find(self, x: int) -> int:
        parent = self.parent
        while parent[x] != x:
            parent[x] = parent[parent[x]]
            x = parent[x]
        return x

    def union(self, x: int, y: int) -> None:
        rx = self.find(x)
        ry = self.find(y)
        if rx == ry:
            return

        parent = self.parent
        rank = self.rank

        if rank[rx] < rank[ry]:
            parent[rx] = ry
        elif rank[rx] > rank[ry]:
            parent[ry] = rx
        else:
            parent[ry] = rx
            rank[rx] += 1


def addr_type(addr: str) -> str:
    """Rough address type from prefix (legacy P2PKH, P2SH, Bech32, Taproot, etc.)."""
    if addr.startswith("1"):
        return "p2pkh"
    if addr.startswith("3"):
        return "p2sh"
    if addr.startswith("bc1q"):
        return "bech32_p2wpkh"
    if addr.startswith("bc1p"):
        return "taproot"
    return "other"


def detect_coinjoin_like(n_in: int, out_values: list[float]) -> bool:
    """
    Very simple CoinJoin-ish heuristic:
    - at least 3 inputs
    - at least 3 outputs
    - at least 3 outputs with exactly the same value
    """
    n_out = len(out_values)
    if n_in < 3 or n_out < 3:
        return False
    cnt = Counter(out_values)
    max_eq = max(cnt.values())
    return max_eq >= 3


# -----------------------------
# Init structures
# -----------------------------
io_paths = sorted(glob.glob(patterns["io"]))
print(f"Found {len(io_paths)} io parquet files.")

uf = UnionFind()
addr_to_id: dict[str, int] = {}  # address -> union-find node index

# RAM-friendly flags indexed by node_id:
# seen_output_flags[i] == 1 if address i was ever seen as a vout
# multi_change_flags[i] bit 0 -> multi-input, bit 1 -> change
seen_output_flags = bytearray()
multi_change_flags = bytearray()

# For coverage / stats
n_txs_total = 0
n_txs_with_multiinput = 0
n_txs_coinjoin_flagged = 0
n_txs_with_change_detected = 0


def ensure_flag_capacity(idx: int) -> None:
    """
    Make sure flag arrays are at least as long as needed for node_id idx.
    """
    needed = idx + 1
    cur = len(seen_output_flags)
    if cur < needed:
        delta = needed - cur
        seen_output_flags.extend(b"\x00" * delta)
        multi_change_flags.extend(b"\x00" * delta)


def get_addr_id(addr: str) -> int:
    """Map address string to union-find index (create if new, plus flag slots)."""
    idx = addr_to_id.get(addr)
    if idx is None:
        idx = uf.make_set()
        addr_to_id[addr] = idx
        ensure_flag_capacity(idx)
    return idx


# -----------------------------
# Main pass over IO parquet files
# -----------------------------
for i, path in enumerate(io_paths, start=1):
    file_day = extract_day_from_path(path)
    if file_day is None:
        print(f"[{i}/{len(io_paths)}] WARNING: could not extract day from path, skipping: {path}")
        continue

    # Skip data before CUTOFF_DAY
    if file_day < CUTOFF_DAY:
        continue

    print(f"[{i}/{len(io_paths)}] Processing {path} (day={file_day}) ...")

    # Read only columns we need (saves RAM)
    df = pl.read_parquet(path, columns=["dir", "txid", "address", "value"])

    # Normalize dir to lowercase UTF-8 strings
    df = df.with_columns(
        pl.col("dir")
        .cast(pl.Utf8)
        .str.to_lowercase()
        .alias("dir")
    )

    # Filter only non-null dir and non-null addresses
    df = (
        df
        .filter(pl.col("dir").is_not_null())
        .filter(pl.col("address").is_not_null())
        .select(["dir", "txid", "address", "value"])
    )

    if df.height == 0:
        del df
        gc.collect()
        continue

    # Optional: print distinct dir values for the first few files to sanity-check encoding
    if i <= 3:
        dir_uniques = df.select(pl.col("dir").unique()).to_series().to_list()
        print(f"  [debug] distinct dir values in this file: {dir_uniques}")

    # --- Optionally ensure every address gets a node_id (singleton by default) ---
    if PRECREATE_NODES_FOR_ALL_ADDRS:
        unique_addrs = df.select(pl.col("address").unique()).to_series().to_list()
        for a in unique_addrs:
            get_addr_id(a)
        del unique_addrs
        gc.collect()

    # Split into inputs and outputs:
    # - outputs: dir in OUTPUT_DIR_VALUES
    # - inputs : everything else
    vin_df = (
        df
        .filter(~pl.col("dir").is_in(list(OUTPUT_DIR_VALUES)))
        .group_by("txid")
        .agg(
            pl.col("address").unique().alias("in_addrs")
        )
    )

    vout_df = (
        df
        .filter(pl.col("dir").is_in(list(OUTPUT_DIR_VALUES)))
        .group_by("txid")
        .agg(
            pl.col("address").alias("out_addrs"),   # keep duplicates if same address repeated
            pl.col("value").alias("out_values")
        )
    )

    # df no longer needed
    del df
    gc.collect()

    if vin_df.height == 0 or vout_df.height == 0:
        # No txs with both inputs and outputs in this file
        del vin_df, vout_df
        gc.collect()
        continue

    # Join inputs & outputs per txid
    tx_df = vin_df.join(vout_df, on="txid", how="inner")

    # free vin/vout to save RAM
    del vin_df, vout_df
    gc.collect()

    if tx_df.height == 0:
        del tx_df
        gc.collect()
        continue

    # Iterate over transactions
    for row in tx_df.iter_rows(named=True):
        in_addrs: list[str] = row["in_addrs"]
        out_addrs: list[str] = row["out_addrs"]
        out_values: list[float] = row["out_values"]

        if not in_addrs or not out_addrs:
            continue

        n_txs_total += 1

        n_in = len(in_addrs)
        n_out = len(out_addrs)

        # CoinJoin/mixer-like detection: skip these txs for heuristics
        is_coinjoin = detect_coinjoin_like(n_in, out_values)
        if is_coinjoin:
            n_txs_coinjoin_flagged += 1
            # still mark outputs as seen
            for a in out_addrs:
                a_id = get_addr_id(a)
                seen_output_flags[a_id] = 1
            continue

        # --- Multi-input heuristic ---
        if n_in >= 2:
            n_txs_with_multiinput += 1

            in_ids = [get_addr_id(a) for a in in_addrs]
            for idx in in_ids:
                multi_change_flags[idx] |= 1  # mark as multi-input

            first_id = in_ids[0]
            for idx in in_ids[1:]:
                uf.union(first_id, idx)

        # --- Change-address heuristic ---
        if n_in >= 1 and n_out >= 2:
            in_types = [addr_type(a) for a in in_addrs]
            type_counts = Counter(in_types)
            majority_type = max(type_counts, key=type_counts.get)

            candidates: list[str] = []
            for a in out_addrs:
                if a in in_addrs:
                    continue
                if addr_type(a) != majority_type:
                    continue
                a_id = get_addr_id(a)
                if seen_output_flags[a_id]:
                    # stricter: favor new output addresses
                    continue
                candidates.append(a)

            if len(candidates) == 1:
                change_addr = candidates[0]
                change_id = get_addr_id(change_addr)
                multi_change_flags[change_id] |= 2  # mark as change

                n_txs_with_change_detected += 1

                first_input_id = get_addr_id(in_addrs[0])
                uf.union(first_input_id, change_id)

        # mark all outputs as seen for future change detection
        for a in out_addrs:
            a_id = get_addr_id(a)
            seen_output_flags[a_id] = 1

    # free tx_df at end of this file
    del tx_df
    gc.collect()

# -----------------------------
# Build final entity mapping
# -----------------------------
print("Finalizing entity mapping (compressing components)...")

n_nodes = len(addr_to_id)
print(f"Number of unique addresses with entity nodes (2013+): {n_nodes}")

if n_nodes == 0:
    print("No addresses ended up with union-find nodes. This likely means either:")
    print("  - No transactions had both input and output addresses detected, and")
    print("  - PRECREATE_NODES_FOR_ALL_ADDRS is False, or")
    print("  - Your 'dir' encoding is such that all rows are treated as outputs.")
else:
    # Map each node -> entity id via connected components
    node_to_entity = [0] * n_nodes
    root_to_entity: dict[int, int] = {}
    next_entity_id = 0

    for node in range(n_nodes):
        root = uf.find(node)
        ent = root_to_entity.get(root)
        if ent is None:
            ent = next_entity_id
            root_to_entity[root] = ent
            next_entity_id += 1
        node_to_entity[node] = ent

    print(f"Number of entities (clusters): {next_entity_id}")

    # Compute cluster sizes directly from node_to_entity
    cluster_size_counter = Counter(node_to_entity)
    n_entities = len(cluster_size_counter)

    # For stats:
    if n_entities <= MAX_ENTITIES_FOR_PLOT:
        cluster_sizes = list(cluster_size_counter.values())
        n_singletons = sum(1 for s in cluster_sizes if s == 1)
        n_large = sum(1 for s in cluster_sizes if s >= 10)
    else:
        # Avoid materializing a giant list just for plotting
        cluster_sizes = None
        n_singletons = sum(1 for s in cluster_size_counter.values() if s == 1)
        n_large = sum(1 for s in cluster_size_counter.values() if s >= 10)

    # Build output lists (address, entity_id)
    addresses_out: list[str] = []
    entity_ids_out: list[int] = []

    for addr, node_id in addr_to_id.items():
        addresses_out.append(addr)
        entity_ids_out.append(node_to_entity[node_id])

    entities_df = pl.DataFrame(
        {
            "address": addresses_out,
            "entity_id": entity_ids_out,
        }
    )

    out_path = OUTPUT_DIR / "entities_multiinput_change_singleton_2013_onward.parquet"
    entities_df.write_parquet(out_path)
    print(f"Saved entity mapping to: {out_path}")

    # We no longer need the address dict; other large structures will be GC'ed soon anyway
    del addr_to_id, node_to_entity, addresses_out, entity_ids_out
    gc.collect()

    # -----------------------------
    # Basic stats & plots
    # -----------------------------
    # Heuristic coverage (address-level)
    n_addrs_total = n_nodes
    # For safety, only consider first n_nodes entries in the flag arrays
    flags_view = multi_change_flags[:n_nodes]

    n_addrs_multi = sum(1 for v in flags_view if (v & 1))
    n_addrs_change = sum(1 for v in flags_view if (v & 2))
    n_addrs_touched = sum(1 for v in flags_view if (v & 3))
    n_addrs_only_singleton = n_addrs_total - n_addrs_touched

    print("Computing cluster size statistics...")

    print(f"Total entities: {n_entities}")
    print(f"Singleton entities (size=1): {n_singletons}")
    print(f"Entities with size >= 10: {n_large}")

    print("\nHeuristic coverage:")
    print(f"  Total txs processed (with both vin & vout): {n_txs_total}")
    print(f"  Txs flagged as CoinJoin-like (skipped): {n_txs_coinjoin_flagged}")
    print(f"  Txs using multi-input heuristic: {n_txs_with_multiinput}")
    print(f"  Txs with change-address detected: {n_txs_with_change_detected}")
    print(f"  Addresses in multi-input txs (unique): {n_addrs_multi}")
    print(f"  Addresses detected as change (unique): {n_addrs_change}")
    print(f"  Addresses only in singleton entities: {n_addrs_only_singleton}")

    # ---- Plot 1: cluster size distribution ----
    if cluster_sizes is not None and any(s > 0 for s in cluster_sizes):
        plt.figure(figsize=(8, 5))
        plt.hist(cluster_sizes, bins=100, log=True)
        plt.xlabel("Cluster size (number of addresses in entity)")
        plt.ylabel("Frequency (log scale)")
        plt.title("Distribution of Bitcoin Entity Cluster Sizes (2013+)")
        plt.tight_layout()
        plt.show()
        plt.close()
    elif cluster_sizes is None:
        print(f"Skipping cluster size histogram (too many entities > {MAX_ENTITIES_FOR_PLOT}).")
    else:
        print("No positive cluster sizes to plot (cluster size histogram skipped).")

    # ---- Plot 2: heuristic coverage (address-level) ----
    if n_addrs_total > 0:
        plt.figure(figsize=(6, 4))
        plt.bar(
            ["multi-input\n(addrs)", "change\n(addrs)", "only\nsingleton"],
            [n_addrs_multi, n_addrs_change, n_addrs_only_singleton],
        )
        plt.ylabel("Number of addresses")
        plt.title("Heuristic Coverage of Addresses (2013+)")
        plt.tight_layout()
        plt.show()
        plt.close()
    else:
        print("No addresses to plot (heuristic coverage bar chart skipped).")


Parquet file counts: {'blocks': 7713, 'txs': 7713, 'io': 7678}
Found 7678 io parquet files.
[1498/7678] Processing /media/vatereal/Main/parquet/io/day=2013-01-01/io-000214563-000214724.parquet (day=2013-01-01) ...
[1499/7678] Processing /media/vatereal/Main/parquet/io/day=2013-01-02/io-000214725-000214877.parquet (day=2013-01-02) ...
[1500/7678] Processing /media/vatereal/Main/parquet/io/day=2013-01-03/io-000214878-000215032.parquet (day=2013-01-03) ...
[1501/7678] Processing /media/vatereal/Main/parquet/io/day=2013-01-03/io-000215033-000215039.parquet (day=2013-01-03) ...
[1502/7678] Processing /media/vatereal/Main/parquet/io/day=2013-01-04/io-000215040-000215192.parquet (day=2013-01-04) ...
[1503/7678] Processing /media/vatereal/Main/parquet/io/day=2013-01-05/io-000215193-000215333.parquet (day=2013-01-05) ...
[1504/7678] Processing /media/vatereal/Main/parquet/io/day=2013-01-06/io-000215334-000215478.parquet (day=2013-01-06) ...
[1505/7678] Processing /media/vatereal/Main/parquet/io